# Target:

Training of [microsoft/Phi-3.5-mini-instruct](https://huggingface.co/microsoft/Phi-3.5-mini-instruct) with (slightly messy) Godot 4.3 documentation.

## Desired Outcome

More reliable Q&A about Godot 4.3 on ```edg3/godot-4.3-phi-3.5-mini-instruct```.

### 🤗 AutoTrain LLM

In order to use this colab:

- upload train.csv to a folder named `data/`
- train.csv must contain a `text` column
- choose a project name if you wish
- change model if you wish, you can use most of the text-generation models from Hugging Face Hub
- add huggingface information (token) if you wish to push trained model to huggingface hub
- update hyperparameters if you wish
- click `Runtime > Run all` or run each cell individually
- they share you can report issues / feature requests here: https://github.com/huggingface/autotrain-advanced/issues

#### Get AutoTrain

In [ ]:
#!pip install -U autotrain-advanced --break-system-packages

In [1]:
!autotrain setup

INFO     | 2024-10-09 14:08:33 | autotrain.cli.run_setup:run:43 - Installing latest xformers
INFO     | 2024-10-09 14:08:44 | autotrain.cli.run_setup:run:45 - Successfully installed latest xformers


In [2]:
import os
from autotrain import __version__
print(f'AutoTrain version: {__version__}')

AutoTrain version: 0.8.21


#### Project Config

Note: if you are using a restricted/private model, you need to enter your Hugging Face token in the next step.

In [ ]:
!git clone --depth 1 https://huggingface.co/microsoft/Phi-3.5-mini-instruct

In [3]:
project_name = 'godot43llama27b' # folder you will create
model_name = 'content/' # where your model is (e.g. 'git clone ...' renamed to 'content/')

#### Push to Hub?

Use these only if you want to push your trained model to a private repo in your Hugging Face Account

If you dont use these, the model will be saved in Google Colab and you are required to download it manually.

Please enter your Hugging Face write token. The trained model will be saved to your Hugging Face account.

You can find your token here: https://huggingface.co/settings/tokens

In [4]:
push_to_hub = False # @param ["False", "True"] {type:"raw"}
hf_token = "hf_XXX" #@param {type:"string"}
hf_username = "abc" #@param {type:"string"}

#### Hyperparameters

In [5]:
unsloth = False             # @param ["False", "True"] {type:"raw"}
learning_rate = 2e-4        # @param {type:"number"}
num_epochs = 1              # @param {type:"number"}
batch_size = 1              # @param {type:"slider", min:1, max:32, step:1}
block_size = 1024           # @param {type:"number"}
trainer = "sft"             # @param ["generic", "sft"] {type:"string"}
warmup_ratio = 0.1          # @param {type:"number"}
weight_decay = 0.01         # @param {type:"number"}
gradient_accumulation = 4   # @param {type:"number"}
mixed_precision = "fp16"    # @param ["fp16", "bf16", "none"] {type:"string"}
peft = True                 # @param ["False", "True"] {type:"raw"}
quantization = "int4"       # @param ["int4", "int8", "none"] {type:"string"}
lora_r = 16                 #@param {type:"number"}
lora_alpha = 32             #@param {type:"number"}
lora_dropout = 0.05         #@param {type:"number"}

#### Setup Environment Vars and Conf.yaml

In [6]:
os.environ["HF_TOKEN"] = hf_token
os.environ["HF_USERNAME"] = hf_username

conf = f"""
task: llm-{trainer}
base_model: {model_name}
project_name: {project_name}
log: tensorboard
backend: local

data:
  path: data/
  train_split: train
  valid_split: null
  chat_template: null
  column_mapping:
    text_column: text

params:
  block_size: {block_size}
  lr: {learning_rate}
  warmup_ratio: {warmup_ratio}
  weight_decay: {weight_decay}
  epochs: {num_epochs}
  batch_size: {batch_size}
  gradient_accumulation: {gradient_accumulation}
  mixed_precision: {mixed_precision}
  peft: {peft}
  quantization: {quantization}
  lora_r: {lora_r}
  lora_alpha: {lora_alpha}
  lora_dropout: {lora_dropout}
  unsloth: {unsloth}

hub:
  username: ${{HF_USERNAME}}
  token: ${{HF_TOKEN}}
  push_to_hub: {push_to_hub}
"""

with open("conf.yaml", "w") as f:
  f.write(conf)

#### Run Training

In [ ]:
#!pip install -q aqlm[gpu,cpu]  --break-system-packages

In [7]:
!autotrain --config conf.yaml

INFO     | 2024-10-09 14:09:31 | autotrain.cli.autotrain:main:60 - Using AutoTrain configuration: conf.yaml
INFO     | 2024-10-09 14:09:32 | autotrain.parser:__post_init__:148 - Running task: lm_training
INFO     | 2024-10-09 14:09:32 | autotrain.parser:__post_init__:149 - Using backend: local
INFO     | 2024-10-09 14:09:32 | autotrain.parser:run:212 - {'model': 'content/', 'project_name': 'godot43llama27b', 'data_path': 'data/', 'train_split': 'train', 'valid_split': None, 'add_eos_token': True, 'block_size': 1024, 'model_max_length': 2048, 'padding': 'right', 'trainer': 'sft', 'use_flash_attention_2': False, 'log': 'tensorboard', 'disable_gradient_checkpointing': False, 'logging_steps': -1, 'eval_strategy': 'epoch', 'save_total_limit': 1, 'auto_find_batch_size': False, 'mixed_precision': 'fp16', 'lr': 0.0002, 'epochs': 1, 'batch_size': 1, 'warmup_ratio': 0.1, 'gradient_accumulation': 4, 'optimizer': 'adamw_torch', 'scheduler': 'linear', 'weight_decay': 0.01, 'max_grad_norm': 1.0, 'se

 10%|████                                   | 100/968 [25:13<3:37:31, 15.04s/it]INFO     | 2024-10-09 14:40:47 | autotrain.trainers.common:on_log:225 - {'loss': 1.6904, 'grad_norm': 0.6797800064086914, 'learning_rate': 0.000199540757749713, 'epoch': 0.10327911179963853}
{'loss': 1.6904, 'grad_norm': 0.6797800064086914, 'learning_rate': 0.000199540757749713, 'epoch': 0.1}
 13%|█████                                  | 125/968 [31:31<3:33:06, 15.17s/it]INFO     | 2024-10-09 14:47:05 | autotrain.trainers.common:on_log:225 - {'loss': 1.4407, 'grad_norm': 0.4424688518047333, 'learning_rate': 0.00019380022962112517, 'epoch': 0.12909888974954814}
{'loss': 1.4407, 'grad_norm': 0.4424688518047333, 'learning_rate': 0.00019380022962112517, 'epoch': 0.13}
 15%|██████                                 | 150/968 [37:49<3:25:45, 15.09s/it]INFO     | 2024-10-09 14:53:24 | autotrain.trainers.common:on_log:225 - {'loss': 1.5903, 'grad_norm': 0.47043392062187195, 'learning_rate': 0.00018805970149253734, 'ep

### Test the Model

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

tokenizer = AutoTokenizer.from_pretrained("godot43llama27b/")
model = AutoModelForCausalLM.from_pretrained(
    "godot43llama27b/", 
    device_map="cuda", 
    torch_dtype="auto", 
    trust_remote_code=True
)
model.resize_token_embeddings(len(tokenizer))

def Query(message):
    messages = [
        {"role": "system", "content": "You are a helpful AI assistant."},
        {"role": "user", "content": message},
    ]

    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
    )

    generation_args = {
        "max_new_tokens": 500,
        "return_full_text": False,
        "temperature": 0.0,
        "do_sample": False,
    }

    output = pipe(messages, **generation_args)
    print(output[0]['generated_text'])

2024-10-10 06:54:55.384390: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-10 06:54:56.023479: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-10 06:54:56.167511: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-10 06:54:57.077188: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-10 06:55:04.281703: W tensorflow/compiler/tf2

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

RuntimeError: Error(s) in loading state_dict for Phi3ForCausalLM:
	size mismatch for model.embed_tokens.weight: copying a param with shape torch.Size([32011, 3072]) from checkpoint, the shape in current model is torch.Size([32064, 3072]).
	size mismatch for lm_head.weight: copying a param with shape torch.Size([32011, 3072]) from checkpoint, the shape in current model is torch.Size([32064, 3072]).

### 6.1: What’s new in Godot 4.3 compared to previous versions?

In [ ]:
Query('What’s new in Godot 4.3 compared to previous versions?')

### 6.2: Can you explain the changes in the rendering pipeline introduced in Godot 4.3?

In [ ]:
Query('Can you explain the changes in the rendering pipeline introduced in Godot 4.3?')

### 6.3: How do you implement a custom shader in Godot 4.3?

In [ ]:
Query('How do you implement a custom shader in Godot 4.3?')

### 6.4: What are the key differences between GDScript and C# in Godot 4.3?

In [ ]:
Query('What are the key differences between GDScript and C# in Godot 4.3?')

### 6.5: How would you optimize a game for performance in Godot 4.3?

In [ ]:
Query('How would you optimize a game for performance in Godot 4.3?')

### 6.6: Can you describe the process of creating a custom node in Godot 4.3?

In [ ]:
Query('Can you describe the process of creating a custom node in Godot 4.3?')

### 6.7: What are the new features in the animation system of Godot 4.3?

In [ ]:
Query('What are the new features in the animation system of Godot 4.3?')

### 6.8: How do you handle input events in Godot 4.3?

In [ ]:
Query('How do you handle input events in Godot 4.3?')

### 6.9: What are the best practices for using the new navigation system in Godot 4.3?

In [ ]:
('What are the best practices for using the new navigation system in Godot 4.3?')

### 6.10: Can you walk me through setting up a multiplayer game in Godot 4.3?

In [ ]:
Query('Can you walk me through setting up a multiplayer game in Godot 4.3?')

### 6.11: How can I use C# to create a procedurally generated 3D chunk in a node in Godot 4.3 on its own?

In [ ]:
Query('How can I use C# to create a procedurally generated 3D chunk in a node in Godot 4.3 on its own?')

### 6.12: How can I make nodes used as chunks, similar to minecraft, to store data in Godot 4.3 so that I can create a synchronous multiplayer world that can be hosted by 1 player, and have a second player connect?

In [ ]:
Query('How can I make nodes used as chunks, similar to minecraft, to store data in Godot 4.3 so that I can create a synchronous multiplayer world that can be hosted by 1 player, and have a second player connect?')

### 6.13: In Godot 4.3, using C#, can you create 2D snake from scratch?

In [ ]:
Query('In Godot 4.3, using C#, can you create 2D snake from scratch?')

# Review (Personal):



In [ ]:
from huggingface_hub import HfApi
from huggingface_hub import create_repo
import os

# Define your model path and repository name
model_path = "godot43llama27b/"
repo_name = "edg3/godot-4.3-phi-3.5-mini-instruct"
my_token = '<your-token-here>' 

if False: # first time true, else repo exists make false
    create_repo(repo_name, token=my_token)

# List all directories inside the model path
folders = [f for f in os.listdir(model_path) if os.path.isdir(os.path.join(model_path, f))]
skip_check = False

# Print the list of folders
if folders and not skip_check:
    print("Folders inside 'model/':")
    for folder in folders:
        print(folder)
else:
    # Initialize the API
    api = HfApi()

    # Upload the model
    api.upload_folder(
        folder_path=model_path,
        repo_id=repo_name,
        commit_message="Second training",
        token=my_token
    )